In [4]:
import argparse
import struct
import torch
import torch.nn as nn
from utils.mobilenetv2 import MobileNetV2
from torchsummary import summary

In [23]:
def masked_fp32(num, n_digits):
    string_bin = format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
    masked = string_bin[:9+n_digits] + '0'*(23-n_digits)
    return struct.unpack('!f',struct.pack('!I', int(masked, 2)))[0]

In [5]:
def float_to_bin(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def bin_to_float(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

In [2]:
n_digits = 5
a = 0.02423
a_bin = float_to_bin(a)
print(a_bin)
masked = a_bin[:9+n_digits] + '0'*(23-n_digits)
print(masked)
result = bin_to_float(masked)
print(result)
print(abs(a - result))

00111100110001100111110111111110
00111100110001000000000000000000
0.02392578125
0.00030421875000000154


In [24]:
x = 0.02423
n = masked_fp32(x, 5)
print(n)

0.02392578125


In [3]:
n_digits = 6
a = 0.02423
a_bin = float_to_bin(a)
print(a_bin)
masked = a_bin[:9+n_digits] + '0'*(23-n_digits)
print(masked)
result = bin_to_float(masked)
print(result)
print(abs(a - result))

00111100110001100111110111111110
00111100110001100000000000000000
0.024169921875
6.0078125000001537e-05


In [39]:
model = MobileNetV2()
model.load_state_dict(torch.load('./data/mobilenet_v2-b0353104.pth'))
model.eval()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_

In [20]:
for i, param in enumerate(model.parameters()):
    print(i)
    print(param.shape)

0
torch.Size([32, 3, 3, 3])
1
torch.Size([32])
2
torch.Size([32])
3
torch.Size([32, 1, 3, 3])
4
torch.Size([32])
5
torch.Size([32])
6
torch.Size([16, 32, 1, 1])
7
torch.Size([16])
8
torch.Size([16])
9
torch.Size([96, 16, 1, 1])
10
torch.Size([96])
11
torch.Size([96])
12
torch.Size([96, 1, 3, 3])
13
torch.Size([96])
14
torch.Size([96])
15
torch.Size([24, 96, 1, 1])
16
torch.Size([24])
17
torch.Size([24])
18
torch.Size([144, 24, 1, 1])
19
torch.Size([144])
20
torch.Size([144])
21
torch.Size([144, 1, 3, 3])
22
torch.Size([144])
23
torch.Size([144])
24
torch.Size([24, 144, 1, 1])
25
torch.Size([24])
26
torch.Size([24])
27
torch.Size([144, 24, 1, 1])
28
torch.Size([144])
29
torch.Size([144])
30
torch.Size([144, 1, 3, 3])
31
torch.Size([144])
32
torch.Size([144])
33
torch.Size([32, 144, 1, 1])
34
torch.Size([32])
35
torch.Size([32])
36
torch.Size([192, 32, 1, 1])
37
torch.Size([192])
38
torch.Size([192])
39
torch.Size([192, 1, 3, 3])
40
torch.Size([192])
41
torch.Size([192])
42
torch.Size([3

In [25]:
for i, param in enumerate(model.parameters()):
    if len(param.shape) == 1:
        for w in range(param.shape[0]):
            param[w] = masked_fp32(param[w], 5)
    if len(param.shape) == 2:
        for w in range(param.shape[0]):
            for x in range(param.shape[1]):
                param[w][x] = masked_fp32(param[w][x], 5)
    if len(param.shape) == 4:
        for w in range(param.shape[0]):
            for x in range(param.shape[1]):
                for y in range(param.shape[2]):
                    for z in range(param.shape[3]):
                        param[w][x][y][z] = masked_fp32(param[w][x][y][z], 5)                  

Parameter containing:
tensor([[[[ 1.3185e-02, -4.3213e-03,  1.4823e-02],
          [ 3.2780e-02, -2.5385e-02,  6.8572e-03],
          [ 1.0549e-02, -3.7347e-02, -1.4727e-02]],

         [[ 7.9917e-03, -5.9146e-03,  1.5076e-02],
          [ 1.9999e-02, -3.2863e-02, -2.0859e-03],
          [ 1.1350e-02, -3.2956e-02, -7.8733e-03]],

         [[-2.5234e-02, -2.0167e-02, -9.9620e-03],
          [-1.1213e-02, -2.9266e-02, -1.5218e-02],
          [-2.6531e-02, -3.3449e-02, -2.4215e-02]]],


        [[[-6.3694e-02, -2.1300e-02,  2.1416e-02],
          [ 1.3037e-01,  3.7967e-01,  4.1983e-02],
          [-1.8793e-01, -2.7921e-01, -3.8335e-02]],

         [[-6.7306e-02,  5.3145e-02,  1.8353e-03],
          [ 2.6662e-01,  7.4863e-01,  1.0166e-01],
          [-3.3641e-01, -6.0844e-01, -7.6828e-02]],

         [[-1.5798e-02, -1.0921e-02,  2.6319e-02],
          [ 4.2062e-02,  1.7312e-01,  1.5455e-03],
          [-6.2128e-02, -1.5648e-01,  1.0293e-03]]],


        [[[-1.0511e-01,  9.7661e-02, -7.8915

In [34]:
for i, param in enumerate(model.parameters()):
    if i == 0:
        print(len(param.shape))

4


In [40]:
for i, param in enumerate(model.parameters()):
    if i == 0:
        print(param[0][0][0][0])
        param[0][0][0][0] = masked_fp32(param[0][0][0][0], 5)
        print(param[0][0][0][0])

tensor(0.0132, grad_fn=<SelectBackward>)
tensor(0.0132, grad_fn=<SelectBackward>)


In [41]:
for i, param in enumerate(model.parameters()):
    if len(param.shape) == 1:
        for w in range(param.shape[0]):
            param[w] = masked_fp32(param[w], 5)
    if len(param.shape) == 2:
        for w in range(param.shape[0]):
            for x in range(param.shape[1]):
                param[w][x] = masked_fp32(param[w][x], 5)
    if len(param.shape) == 4:
        for w in range(param.shape[0]):
            for x in range(param.shape[1]):
                for y in range(param.shape[2]):
                    for z in range(param.shape[3]):
                        param[w][x][y][z] = masked_fp32(param[w][x][y][z], 5)

In [42]:
for i, param in enumerate(model.parameters()):
    print(param)

Parameter containing:
tensor([[[[ 1.3184e-02, -4.2725e-03,  1.4648e-02],
          [ 3.2227e-02, -2.4902e-02,  6.8359e-03],
          [ 1.0498e-02, -3.7109e-02, -1.4648e-02]],

         [[ 7.8125e-03, -5.8594e-03,  1.4893e-02],
          [ 1.9531e-02, -3.2227e-02, -2.0752e-03],
          [ 1.1230e-02, -3.2227e-02, -7.8125e-03]],

         [[-2.4902e-02, -2.0020e-02, -9.7656e-03],
          [-1.0986e-02, -2.8809e-02, -1.5137e-02],
          [-2.6367e-02, -3.3203e-02, -2.3926e-02]]],


        [[[-6.2500e-02, -2.0996e-02,  2.0996e-02],
          [ 1.2891e-01,  3.7500e-01,  4.1016e-02],
          [-1.8750e-01, -2.7344e-01, -3.8086e-02]],

         [[-6.6406e-02,  5.2734e-02,  1.8311e-03],
          [ 2.6562e-01,  7.3438e-01,  1.0156e-01],
          [-3.3594e-01, -5.9375e-01, -7.6172e-02]],

         [[-1.5625e-02, -1.0742e-02,  2.5879e-02],
          [ 4.1992e-02,  1.7188e-01,  1.5259e-03],
          [-6.1523e-02, -1.5625e-01,  1.0071e-03]]],


        [[[-1.0352e-01,  9.7656e-02, -7.8125

Parameter containing:
tensor([[[[ 0.1016]],

         [[-0.1035]],

         [[ 0.1445]],

         ...,

         [[ 0.0322]],

         [[-0.0115]],

         [[ 0.1992]]],


        [[[-0.0645]],

         [[ 0.0410]],

         [[-0.0332]],

         ...,

         [[ 0.0684]],

         [[ 0.1133]],

         [[ 0.0664]]],


        [[[ 0.1230]],

         [[ 0.0664]],

         [[-0.0547]],

         ...,

         [[ 0.2344]],

         [[-0.0254]],

         [[ 0.1328]]],


        ...,


        [[[ 0.0200]],

         [[ 0.2031]],

         [[ 0.1953]],

         ...,

         [[ 0.0051]],

         [[-0.0298]],

         [[-0.1035]]],


        [[[-0.2891]],

         [[ 0.0840]],

         [[-0.0957]],

         ...,

         [[ 0.0957]],

         [[ 0.0781]],

         [[-0.0762]]],


        [[[-0.0420]],

         [[ 0.0303]],

         [[-0.1113]],

         ...,

         [[-0.0161]],

         [[ 0.3594]],

         [[-0.1074]]]], grad_fn=<CopySlices>)
Parameter co

Parameter containing:
tensor([-2.9297e-02, -1.2109e-01, -2.9297e-02,  1.8555e-02, -6.2500e-02,
        -2.9785e-02,  1.1914e-01,  1.4404e-02, -3.9062e-02, -4.1016e-02,
        -2.6367e-02, -8.5938e-02, -8.9844e-02, -1.1914e-01,  1.6113e-02,
        -6.6406e-02,  1.4404e-02,  1.9141e-01,  5.2734e-02,  1.1523e-01,
        -9.5703e-02, -2.5781e-01, -8.7891e-02,  1.7188e-01, -1.0547e-01,
        -2.3828e-01,  7.2266e-02, -1.4062e-01,  8.0078e-02, -8.5938e-02,
        -6.2500e-02,  1.6016e-01, -6.4453e-02, -2.6855e-02,  5.4688e-02,
        -5.0781e-02,  6.8359e-02, -8.7891e-02,  1.0547e-01,  1.7188e-01,
        -9.5703e-02,  1.9531e-02,  6.2500e-02, -4.1016e-02,  4.7852e-02,
         2.3926e-02, -1.2891e-01, -1.6797e-01, -5.0000e-01, -1.0352e-01,
        -1.1328e-01,  1.6113e-02,  3.3203e-02, -1.8311e-04, -7.2266e-02,
        -6.1523e-02, -1.7578e-02,  3.9062e-02, -3.1250e-02, -2.0996e-02,
        -4.1992e-02,  6.2256e-03, -1.2500e-01, -1.4453e-01, -4.7852e-02,
        -3.5156e-02, -8.7891e

Parameter containing:
tensor([0.0996, 0.1191, 0.0938, 0.1445, 0.1953, 0.1250, 0.1328, 0.0781, 0.0938,
        0.1523, 0.1797, 0.1250, 0.1133, 0.1074, 0.1074, 0.1094, 0.0820, 0.1406,
        0.0957, 0.1250, 0.1250, 0.1074, 0.1445, 0.1094, 0.1172, 0.1250, 0.1230,
        0.0957, 0.1641, 0.1484, 0.0898, 0.1914, 0.1016, 0.1875, 0.0723, 0.0977,
        0.1875, 0.1289, 0.1055, 0.1328, 0.1406, 0.1211, 0.1289, 0.1289, 0.1680,
        0.1152, 0.1250, 0.1445, 0.1719, 0.1836, 0.1367, 0.1523, 0.1797, 0.1914,
        0.1992, 0.0859, 0.0977, 0.1484, 0.0762, 0.0859, 0.1484, 0.1680, 0.1133,
        0.1367, 0.1250, 0.0918, 0.1719, 0.2109, 0.1152, 0.1562, 0.1191, 0.1094,
        0.1367, 0.2266, 0.0957, 0.1445, 0.1562, 0.1250, 0.1914, 0.1602, 0.0019,
        0.1328, 0.1230, 0.1250, 0.1250, 0.1250, 0.1523, 0.1016, 0.1211, 0.1094,
        0.1094, 0.1250, 0.1250, 0.1445, 0.1094, 0.1641, 0.1562, 0.1055, 0.1797,
        0.0957, 0.1289, 0.1230, 0.1367, 0.1797, 0.1406, 0.1250, 0.1055, 0.1641,
        0.1016, 0.

In [43]:
torch.save(model.state_dict(), './data/masked.pth')

In [45]:
m = MobileNetV2()
m.load_state_dict(torch.load('./data/masked.pth'))
m.eval()
for i, param in enumerate(m.parameters()):
    print(param)

Parameter containing:
tensor([[[[ 1.3184e-02, -4.2725e-03,  1.4648e-02],
          [ 3.2227e-02, -2.4902e-02,  6.8359e-03],
          [ 1.0498e-02, -3.7109e-02, -1.4648e-02]],

         [[ 7.8125e-03, -5.8594e-03,  1.4893e-02],
          [ 1.9531e-02, -3.2227e-02, -2.0752e-03],
          [ 1.1230e-02, -3.2227e-02, -7.8125e-03]],

         [[-2.4902e-02, -2.0020e-02, -9.7656e-03],
          [-1.0986e-02, -2.8809e-02, -1.5137e-02],
          [-2.6367e-02, -3.3203e-02, -2.3926e-02]]],


        [[[-6.2500e-02, -2.0996e-02,  2.0996e-02],
          [ 1.2891e-01,  3.7500e-01,  4.1016e-02],
          [-1.8750e-01, -2.7344e-01, -3.8086e-02]],

         [[-6.6406e-02,  5.2734e-02,  1.8311e-03],
          [ 2.6562e-01,  7.3438e-01,  1.0156e-01],
          [-3.3594e-01, -5.9375e-01, -7.6172e-02]],

         [[-1.5625e-02, -1.0742e-02,  2.5879e-02],
          [ 4.1992e-02,  1.7188e-01,  1.5259e-03],
          [-6.1523e-02, -1.5625e-01,  1.0071e-03]]],


        [[[-1.0352e-01,  9.7656e-02, -7.8125

Parameter containing:
tensor([0.1289, 0.1172, 0.1562, 0.1094, 0.1367, 0.1445, 0.1484, 0.1211, 0.1289,
        0.0781, 0.0488, 0.1562, 0.1328, 0.1562, 0.1680, 0.1367, 0.1445, 0.1152,
        0.1445, 0.1484, 0.1445, 0.1445, 0.1484, 0.1602, 0.1230, 0.1113, 0.1113,
        0.1445, 0.1602, 0.1445, 0.1113, 0.0840, 0.1328, 0.1035, 0.1367, 0.0801,
        0.1406, 0.1074, 0.1562, 0.1758, 0.1484, 0.1367, 0.0615, 0.1250, 0.1523,
        0.1367, 0.1328, 0.1152, 0.1250, 0.1562, 0.0703, 0.1250, 0.1250, 0.1152,
        0.1094, 0.0918, 0.1230, 0.1523, 0.0879, 0.0723, 0.1211, 0.1367, 0.0742,
        0.1328, 0.1406, 0.1172, 0.1016, 0.1074, 0.1562, 0.0723, 0.1367, 0.1914,
        0.0498, 0.1211, 0.1445, 0.1191, 0.1289, 0.1523, 0.1484, 0.1406, 0.1484,
        0.1836, 0.1562, 0.1875, 0.1406, 0.1562, 0.1875, 0.1797, 0.1562, 0.1328,
        0.1328, 0.1562, 0.1484, 0.1406, 0.1836, 0.1230, 0.1484, 0.1445, 0.1406,
        0.1641, 0.1289, 0.1289, 0.1230, 0.1523, 0.1289, 0.1191, 0.1445, 0.1602,
        0.1367, 0.

Parameter containing:
tensor([-3.4180e-02, -1.2207e-02,  3.6133e-02,  1.2939e-02, -7.2266e-02,
         3.6011e-03,  1.9531e-02, -2.2461e-02,  2.0312e-01, -1.9531e-02,
        -3.3203e-02,  7.0312e-02,  3.4180e-02,  1.9226e-03,  7.8125e-02,
         3.1250e-02,  1.9141e-01,  5.0781e-02,  4.0039e-02,  6.2500e-02,
         6.4453e-02,  8.9844e-02,  4.4250e-04,  4.8828e-03,  6.8359e-02,
         2.8809e-02,  1.4844e-01,  1.5234e-01, -1.2695e-02,  1.7188e-01,
        -2.2461e-02, -3.3203e-02,  1.0547e-01, -5.0781e-02,  1.2109e-01,
        -5.4688e-02,  2.3438e-02, -2.8809e-02,  8.2031e-02, -8.3984e-02,
        -4.1992e-02,  8.9844e-02, -2.9785e-02, -6.1523e-02, -3.8086e-02,
        -5.8594e-02,  4.6875e-02,  7.0312e-02,  1.8555e-02, -2.6855e-02,
         6.6406e-02, -7.0801e-03, -3.4180e-02, -6.8359e-02, -6.6406e-02,
         1.0742e-01,  8.9844e-02,  3.1250e-02,  1.9531e-01, -5.1758e-02,
        -5.7617e-02, -1.4038e-03,  2.9785e-02, -2.9297e-02, -5.6641e-02,
         8.0078e-02, -3.7109e